<a href="https://colab.research.google.com/github/vishal-burman/PyTorch-Architectures/blob/master/modeling_BPR/test_sample_BPR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! wget https://raw.githubusercontent.com/RUCAIBox/RecBole/master/recbole/dataset_example/ml-100k/ml-100k.inter

In [41]:
import pandas as pd
import numpy as np
import scipy.sparse as sp

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [9]:
dataset = pd.read_csv('ml-100k.inter', sep='\t')

In [10]:
dataset.head()

,user_id:token,item_id:token,rating:float,timestamp:float
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [13]:
user_num = dataset['user_id:token'].max() + 1
item_num = dataset['item_id:token'].max() + 1

In [15]:
train_data = dataset.values.tolist()

In [20]:
train_mat = sp.dok_matrix((user_num, item_num), dtype=np.float32)

In [23]:
for x in train_data:
  train_mat[x[0], x[1]] = 1.0

In [31]:
features_fill = []
for x in train_data:
  u, i = x[0], x[1]
  for t in range(4):
    j = np.random.randint(item_num)
    while (u, j) in train_mat:
      j = np.random.randint(item_num)
    features_fill.append([u, i, j])

In [54]:
class RecDataset(Dataset):
  def __init__(self, features, num_item, train_mat=None, num_ng=4, is_training=False):
    self.features = features
    self.num_item = num_item
    self.train_mat = train_mat
    self.num_ng = num_ng
    self.is_training = is_training
    if self.is_training:
      self.negative_sample()
    
  def negative_sample(self):
    assert self.is_training, 'no need to sample while testing'
    self.features_fill = []
    for x in self.features:
      u, i = x[0], x[1]
      for t in range(self.num_ng):
        j = np.random.randint(self.num_item)
        while (u, j) in self.train_mat:
          j = np.random.randint(self.num_item)
        self.features_fill.append([u, i, j])
  
  def __len__(self):
    return self.num_ng * len(self.features) if self.is_training else len(self.features)
  
  def __getitem__(self, idx):
    features = self.features_fill if self.is_training else self.features
    user = self.features[idx][0]
    item_pos = self.features[idx][1]
    item_neg = self.features[idx][2] if self.is_training else self.features[idx][1]
    return {
        'user_id': torch.tensor(user),
        'pos_item_id': torch.tensor(item_pos),
        'neg_item_id': torch.tensor(item_neg),
    }

In [55]:
dataset = RecDataset(train_data, num_item=item_num, train_mat=train_mat, is_training=True)

In [59]:
loader = DataLoader(dataset, shuffle=False, batch_size=2)

torch.Size([2])
